## Identification of interesting VIIRS files

This notebook process a general bash file that retrieve VIIRS images to return a new bash file with the images that mathches ICESat-2 data in a given temporal window

In [1]:
#import icepyx

In [1]:
import pandas as pd
from icepyx import query as ipd

In [3]:
# path with VIIRS's retrieval files
path_viirs_retrieve = "data/VIIRS_bash/"
# name of the original bash file to retrieve the VIIRS data
exe_old = "viirs_bash_original_p68m50p73m45.sh"
# name of the output bash file 
exe_new = "viirs_bash_modified_p68m50p73m45.sh"

In [4]:
# Spatial extend for the ICESat-2 search 
# Observation: in case that the VIIRS file retrieve images in a different spatial extent, then we are no going to have matches, 
# independently of the window. 
#spatial_extent = [-50, 70, -49, 71]
spatial_extent = [-50, 68, -45, 73]

# Temporal window in minutes
minutes = 15
hr = minutes / 60 
# Maximim number of files we are interested in retrieve 
max_viirs = 100000000

In [5]:
file = open(path_viirs_retrieve + exe_old, 'r')
new_file = open(path_viirs_retrieve + exe_new, 'x')

viirs_names = []

counter = 0
counter_viirs = 0

for line in file:
    
    #print(line)
    
    if ("https://ladsweb.modaps.eosdis.nasa.gov" in line) and (len(line) == 146):
                
        if len(viirs_names) < max_viirs:

            Vfile = line[:-1]

            f_t = int(Vfile[-33:-26]+Vfile[-25:-21])
            f_t = pd.to_datetime(f_t, format="%Y%j%H%M")

            # Temporal search window for 
            start = f_t - pd.DateOffset(hours=hr)
            end   = f_t + pd.DateOffset(hours=hr)

            start_date_str = start.strftime('%Y-%m-%d')
            end_date_str   = end.strftime('%Y-%m-%d')
            start_time_str = start.strftime('%H:%M:%S')
            end_time_str   = end.strftime('%H:%M:%S')

            try:

                region_a = ipd.Query("ATL06", spatial_extent, [start_date_str, end_date_str], start_time_str, end_time_str)  # This was different before
                avail_granules = region_a.avail_granules(ids=True)

                viirs_names.append(Vfile)
                new_file.write(Vfile + "\n") # add \n
                assert len(avail_granules) > 0
                counter_viirs += 1

            except AssertionError:
                None
                
            counter += 1
        
        if counter % 50 == 0:
            print(">>> There are a total of", counter_viirs, "viirs files founded out of", counter, "file names.")
        
    else:
        
        new_file.write(line)
        
    
new_file.close()

>>> There are a total of 2 viirs files founded out of 50 file names.
>>> There are a total of 2 viirs files founded out of 100 file names.
>>> There are a total of 3 viirs files founded out of 150 file names.
>>> There are a total of 3 viirs files founded out of 200 file names.
>>> There are a total of 3 viirs files founded out of 250 file names.
>>> There are a total of 4 viirs files founded out of 300 file names.
>>> There are a total of 4 viirs files founded out of 350 file names.
>>> There are a total of 5 viirs files founded out of 400 file names.
>>> There are a total of 6 viirs files founded out of 450 file names.
>>> There are a total of 6 viirs files founded out of 500 file names.
>>> There are a total of 6 viirs files founded out of 550 file names.
>>> There are a total of 7 viirs files founded out of 600 file names.
>>> There are a total of 7 viirs files founded out of 650 file names.
>>> There are a total of 7 viirs files founded out of 700 file names.
>>> There are a total

In [3]:
Took a quick look at a VIIRS file — time seems to be there: A= xarray.open_dataset('/Users/jonathantaylor/Downloads/VNP01_NRT.A2020218.1600.001.nc')     ...: ; A.time_coverage_end                                                                    Out[11]: '2020-08-05T16:06:00.000Z'

1